In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [2]:
import tensorflow_datasets as tfds

imdb,info = tfds.load("imdb_reviews",with_info=True,as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ2W5T4/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ2W5T4/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ2W5T4/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

train_data,test_data = imdb["train"],imdb["test"]

In [0]:
training_sentences = []

training_labels = []

testing_sentences = []

testing_labels = []

for s,l in train_data:

  training_sentences.append(str(s.numpy()))

  training_labels.append(l.numpy())


for s,l in test_data:

  testing_sentences.append(str(s.numpy()))

  testing_labels.append(l.numpy())

In [11]:
print(training_sentences[0])
print(training_labels[0])

training_labels = np.array(training_labels)

testing_labels = np.array(testing_labels)


b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0


In [8]:
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size,oov_token = oov_tok)

tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)

training_padded = pad_sequences(training_sequences,maxlen=max_length,truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

testing_padded = pad_sequences(testing_sequences,maxlen=max_length,truncating=trunc_type)

print(training_padded[0])

[   0    0   59   12   14   35  439  400   18  174   29    1    9   33
 1378 3401   42  496    1  197   25   88  156   19   12  211  340   29
   70  248  213    9  486   62   70   88  116   99   24 5740   12 3317
  657  777   12   18    7   35  406 8228  178 2477  426    2   92 1253
  140   72  149   55    2    1 7525   72  229   70 2962   16    1 2880
    1    1 1506 4998    3   40 3947  119 1608   17 3401   14  163   19
    4 1253  927 7986    9    4   18   13   14 4200    5  102  148 1237
   11  240  692   13   44   25  101   39   12 7232    1   39 1378    1
   52  409   11   99 1214  874  145   10]


In [9]:
from keras.models import Sequential
from keras.layers import Embedding,Dense,Flatten

model = Sequential()

model.add(Embedding(vocab_size,embedding_dim,input_length = max_length))

model.add(Flatten())

model.add(Dense(6,activation = "relu"))

model.add(Dense(1,activation = "sigmoid"))

model.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ["accuracy"])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(training_padded,training_labels,epochs=10,batch_size = 32,validation_data=(testing_padded,testing_labels))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5s 200us/step - loss: 0.4944 - accuracy: 0.7400 - val_loss: 0.3783 - val_accuracy: 0.8308
Epoch 2/10
25000/25000 [==============================] - 3s 130us/step - loss: 0.2354 - accuracy: 0.9120 - val_loss: 0.4132 - val_accuracy: 0.8201
Epoch 3/10
25000/25000 [==============================] - 3s 127us/step - loss: 0.0847 - accuracy: 0.9785 - val_loss: 0.5339 - val_accuracy: 0.8032
Epoch 4/10
25000/25000 [==============================] - 3s 128us/step - loss: 0.0201 - accuracy: 0.9976 - val_loss: 0.6101 - val_accuracy: 0.8039
Epoch 5/10
25000/25000 [==============================] - 3s 130us/step - loss: 0.0048 - accuracy: 0.9996 - val_loss: 0.6870 - val_accuracy: 0.8053
Epoch 6/10
25000/25000 [==============================] - 3s 130us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.7407 - val_accuracy: 0.8084
Epoch 7/10
25000/25000 [==============================] - 3s 1

In [16]:
test = ["the movie was pathetic and a complete waste of time and money"]

test_sequence = tokenizer.texts_to_sequences(test)

test_padded = pad_sequences(test_sequence,maxlen=max_length,truncating=trunc_type)


prediction = model.predict(test_padded)

if prediction >= 0.5:
  print("good review")
else:
  print("bad review")

bad review
